# Generate the NonLinLoc config files
This notebook generate the necessary input file for NLL Vel2Grid and Grid2Time (config.in) using the output of the notebook 3_velocity_to_poly_nnl

Webase the input files on the two templates files:
- nlloc_runfile_template.in 
- ttgrid_runfile_template.in

Based on the work of Loïs Papin with edits from Daniel Trugman

NonLinLoc: {cite:t}`Lomax2000`, {cite:t}`Lomax2009`

In [1]:
import os
import json

## set of input parameter
* poly_id: id of the polygon (int)
* wave: One at a time for the grids calculation, P or S
* out_name: for VGOUT and time tables

In [2]:
# create output
os.makedirs(f'./out_NLL_config_files', exist_ok=True)

# define polygon
poly_id = 2

# one file for each phase
wave = "P"
out_fileP = f'./out_NLL_config_files/ttgrid_runfile_poly{poly_id}_{wave}.in'
wave = "S"
out_fileS = f'./out_NLL_config_files/ttgrid_runfile_poly{poly_id}_{wave}.in'

# params
out_name = "delph"
# with open("cascadia_poly.json", 'r+') as file:
with open("cascadia_poly.json", 'r+') as file:
    file_data = json.load(file)
poly_param = file_data[poly_id-1]
poly_param

{'poly_id': 2.0,
 'olon': -121.3,
 'olat': 48.0,
 'plat1': 46.9,
 'plat2': 49.1,
 'xmin': -124.0,
 'xmax': 124.0,
 'xsep': 1.0,
 'ymin': -124.0,
 'ymax': 124.0,
 'ysep': 1.0,
 'zmin': -4.0,
 'zmax': 100.0,
 'zsep': 1.0,
 'minlon': -122.6,
 'maxlon': -120.0,
 'minlat': 46.9,
 'maxlat': 49.1}

## Generic control file statements
Set transform

In [3]:
def edit_trans_cmd(nll_cmd_lines, poly_param):
    # Find and replace the TRANS line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('TRANS'):
            nll_cmd_lines[i] = f"TRANS LAMBERT WGS-84 {poly_param['olat']} {poly_param['olon']} {poly_param['plat1']} {poly_param['plat2']} 0.0" + ' # modified' + '\n'
            return nll_cmd_lines

In [4]:
# Read the template file
with open('./templates_NLL/ttgrid_runfile_template.in', 'r') as file:
    nll_cmd_lines = file.readlines()

# modify TRANS
nll_cmd_lines = edit_trans_cmd(nll_cmd_lines, poly_param)

# Write the modified content back to a new files
for out_file in [out_fileP, out_fileS]:
    with open(out_file, 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in nll_cmd_lines)
    print('TRANS lines have been modified successfully.')

TRANS lines have been modified successfully.
TRANS lines have been modified successfully.


## Vel2Grid3D

*Given an existing 3D velocity model defined by interpolation between control point nodes and created by velocity inversion programs such as SimulPS, Simul2000 and FDTomo to a 3D model grid. (outputs a 3D Grid) , Vel2Grid3D generates a 3D model Grid header and buffer files containing velocities, slownesses or other model specification.*

command lines for Vel2Grid:
* VGGRID
* VGTYPE
* VGOUT
* VGINP

In [5]:
def edit_vggrid_cmd(nll_cmd_lines, poly_param):
    # Find and replace the VGGRID line in the .run file
    x_num = int(((poly_param['xmax'] - poly_param['xmin'])//poly_param['xsep'])+1)
    y_num = int(((poly_param['ymax'] - poly_param['ymin'])//poly_param['ysep'])+1)
    z_num = int(((poly_param['zmax'] - poly_param['zmin'])//poly_param['zsep'])+1)
    # Syntax 1: VGGRID xNum yNum zNum xOrig yOrig zOrig dx dy dz gridType
    vggrid_command = f"VGGRID {x_num} {y_num} {z_num} {poly_param['xmin']} {poly_param['ymin']} {poly_param['zmin']}"\
                     f" {poly_param['xsep']} {poly_param['ysep']} {poly_param['zsep']} SLOW_LEN"
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('VGGRID'):
            nll_cmd_lines[i] = vggrid_command + ' # modified' + '\n'
            return nll_cmd_lines
        
def edit_vgtype_cmd(nll_cmd_lines, wave):
    # Find and replace the VGTYPE line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('VGTYPE'):
            nll_cmd_lines[i] = f"VGTYPE {wave}" + ' # modified' + '\n'
            return nll_cmd_lines
        
def edit_vgout_cmd(nll_cmd_lines, out_name, poly_id):
    # Find and replace the VGOUT line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('VGOUT'):
            nll_cmd_lines[i] = f"VGOUT ./model/{out_name}_poly{poly_id}_lcc" + ' # modified' + '\n'
            return nll_cmd_lines
        
def edit_vginp_cmd(nll_cmd_lines, input_filename_simulPS):
    vginp_command = 'VGINP ' + input_filename_simulPS + ' SIMUL2K 0 0 0'
    # Find and replace the VGINP line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('#VGINP'):
            nll_cmd_lines[i] = vginp_command + ' # modified' + '\n'
            return nll_cmd_lines

In [6]:
# Read the previously modified .run file

# one each for P and S
for out_file in [out_fileP, out_fileS]:

    # extract wave type
    wave = out_file[-4]

    # open existing file
    with open(out_file, 'r') as file:
        nll_cmd_lines = file.readlines()
    
    # modify VGGRID
    nll_cmd_lines = edit_vggrid_cmd(nll_cmd_lines, poly_param)
    
    # modify VGTYPE
    nll_cmd_lines = edit_vgtype_cmd(nll_cmd_lines, wave)
    
    # modify VGOUT
    nll_cmd_lines = edit_vgout_cmd(nll_cmd_lines, out_name, poly_id)
    
    # modify VGINP
    input_filename_simulPS = f'../data/velocity_models/simulPS/delph_simulps_poly{poly_id}_lcc_200R_{wave}.txt' # Location of the PS velocity models from nb3
    nll_cmd_lines = edit_vginp_cmd(nll_cmd_lines, input_filename_simulPS)
    
    # Write the modified content back to a new file (or overwrite the original)
    with open(out_file, 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in nll_cmd_lines)
    print('VGGRID, VGTYPE, VGOUT, VGINP lines have been modified successfully.')

VGGRID, VGTYPE, VGOUT, VGINP lines have been modified successfully.
VGGRID, VGTYPE, VGOUT, VGINP lines have been modified successfully.


## Grid2Time

*Given a velocity model grid, Grid2Time calculates the travel-time from a source point in a 3D grid to all other points in the grid. Optionally, the program also estimates the take-off angles for rays from each point in the grid to the source.*

Commands to edit:
* INCLUDE : used in place of multiple GTSOURCE, coordinates of the stations that will be used for the locations.
* GTFILES

In [7]:
def edit_include_cmd(nll_cmd_lines, input_filename_stas):
    include_command = 'INCLUDE ' + input_filename_stas
    # Find and replace the INCLUDE line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('INCLUDE'):
            nll_cmd_lines[i] = include_command + ' # modified' + '\n'
            return nll_cmd_lines
        
def edit_gtfiles_cmd(nll_cmd_lines, out_name, poly_id, wave):
    # Find and replace the INCLUDE line in the .run file
    for i, line in enumerate(nll_cmd_lines):
        if line.startswith('GTFILES'):
            nll_cmd_lines[i] = f"GTFILES ./model/{out_name}_poly{poly_id}_lcc ./time/{out_name}_poly{poly_id}_lcc {wave}" + ' # modified' + '\n'
            return nll_cmd_lines

In [8]:
# Read the previously modified .run file

# one each for P and S
for out_file in [out_fileP, out_fileS]:

    # extract wave type
    wave = out_file[-4]

    # read existing file
    with open(out_file, 'r') as file:
        nll_cmd_lines = file.readlines()
    
    # modify INCLUDE for station location
    # Determine the .txt file for station coordinates
    input_filename_stas = f'./stations_poly/stations_poly{poly_id}.txt'
    nll_cmd_lines = edit_include_cmd(nll_cmd_lines, input_filename_stas)
    
    # modify GTFILES
    nll_cmd_lines = edit_gtfiles_cmd(nll_cmd_lines, out_name, poly_id, wave)
            
    # Write the modified content back to a new file (or overwrite the original)
    with open(out_file, 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in nll_cmd_lines)
    
    print('INCLUDE and GTFILES lines have been added successfully.')

INCLUDE and GTFILES lines have been added successfully.
INCLUDE and GTFILES lines have been added successfully.


## verify config file for Vel2Grid and Grid2Time

In [9]:
# What does it look like after modifying the .run file for the commands

# one each for P and S
for out_file in [out_fileP, out_fileS]:
    with open(out_file, 'r') as file:
        lines = file.readlines()
        non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
        for line in non_comment_lines:
            print(line)
    print()

CONTROL 1 54321
TRANS LAMBERT WGS-84 48.0 -121.3 46.9 49.1 0.0 # modified
VGOUT ./model/delph_poly2_lcc # modified
VGTYPE P # modified
VGGRID 249 249 105 -124.0 -124.0 -4.0 1.0 1.0 1.0 SLOW_LEN # modified
VGINP ../data/velocity_models/simulPS/delph_simulps_poly2_lcc_200R_P.txt SIMUL2K 0 0 0 # modified
GTFILES ./model/delph_poly2_lcc ./time/delph_poly2_lcc P # modified
GTMODE GRID3D ANGLES_NO
INCLUDE ./stations_poly/stations_poly2.txt # modified
GT_PLFD  1.0e-3  0

CONTROL 1 54321
TRANS LAMBERT WGS-84 48.0 -121.3 46.9 49.1 0.0 # modified
VGOUT ./model/delph_poly2_lcc # modified
VGTYPE S # modified
VGGRID 249 249 105 -124.0 -124.0 -4.0 1.0 1.0 1.0 SLOW_LEN # modified
VGINP ../data/velocity_models/simulPS/delph_simulps_poly2_lcc_200R_S.txt SIMUL2K 0 0 0 # modified
GTFILES ./model/delph_poly2_lcc ./time/delph_poly2_lcc S # modified
GTMODE GRID3D ANGLES_NO
INCLUDE ./stations_poly/stations_poly2.txt # modified
GT_PLFD  1.0e-3  0

